This notebook is a part of the work 'Nucleation patterns of polymer crystals analyzed by machine learning models' and is written by Atmika Bhardwaj (bhardwaj@ipfdd.de).

The following machine learning (ML) models are employed: AutoEncoders (AE), Hierarchical Clustering (HC), Gaussian Mixture Models (GMM), Multilayer Perceptron (MLP) to harness the capabilities of ML algorithms to unfold the details associated with the different phases that emerge during polymer crystallization. We start by reading a coordinate file (dump file from LAMMPS) and analyze it to study the local environmental information of every coarse-grained bead. Then, we train an AE to compress that information and train a GMM on the compressed data to classify each coarse-grained into amorphous or crytalline depending on its enviromental fingerprint.

This notebook imports all the functions from another file called all_functions.py


In [ ]:
import glob
import random
import pickle
import numpy                  as     np
import pandas                 as     pd
from   numpy.random           import seed
import matplotlib.pyplot      as     plt
import h5py                   as     h5py
from   mpl_toolkits.mplot3d   import Axes3D
from   natsort                import natsorted
from   scipy.spatial          import distance
import scipy.linalg           as linlag
from   all_functions          import *
import control_parameters
param = control_parameters.get_defaults()
seed(1)
plt.rcParams["figure.figsize"]  = (6,5)
plt.rcParams["figure.dpi"]      = 300
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

skip_processing=True

In [ ]:
chainLength     = param["chainlength"]
dx              = param["dR"]
Rmax            = param["Rmax"]
g               = natsorted(glob.glob('../../01_raw_data/dump_files/dump.*'))
filename        = '../../02_processed_data/Labels/AE_labels.sav'
all_ae_labels   = np.array(pickle.load(open(filename, 'rb')))
timestep        = -1
obj             = ReadCoordinates(g[timestep], chainLength)

In [ ]:
for i in np.arange(len(g)):
    aelabels  = all_ae_labels[i]
    class1    = np.where(aelabels == 1)[0]
    print(i, len(class1))

In [ ]:
def recursion(new_indices, class1, grid):
    new_n    = []
    for l in new_indices:
        temp = list(grid.indices[int(np.where(class1 == l)[0])]) #to get the indices, we need the poisiton of l in class1
        new_n.append(temp)                                       #now this contains again original indices of particles wrt 1M
    return new_n

def flatten(x):
    if isinstance(x, collections.abc.Iterable):
        return [a for i in x for a in flatten(i)]
    else:
        return [x]

In [ ]:
def get_domains(timestep, cutoff, all_labels):
    aelabels  = all_labels[timestep]
    class1    = np.where(aelabels == 1)[0]
    
    obj  = ReadCoordinates(g[timestep], chainLength)
    grid = GridSearch(obj, numSigma = 5.0)
    grid.updateNeighborLists(obj, class1, cutoff)
    
    targeted = np.arange(len(class1))
    domains  = []
    visited  = []

    for i in targeted:
        if class1[i] not in visited:
            temp_n      = list(grid.indices[i])
            temp_n      = set(temp_n) - set(visited)                #removing the elements already visited
            temp_n      = list(set(temp_n).intersection(class1))    #keeping the elements that are in class1 only
            neigh       = temp_n.copy()
            visited.append(temp_n)
            visited     = list(flatten(visited))
            n_added     = len(temp_n)

            while n_added != 0:
                temp_n  = recursion(temp_n, class1, grid)
                temp_n  = flatten(temp_n)
                temp_n  = set(temp_n) - set(visited)
                temp_n  = list(temp_n.intersection(class1))
                n_added = len(temp_n)
                #print(n_added)
                neigh.append(temp_n)
                visited.append(temp_n)
                visited = list(set(flatten(visited)))
                #print('n_visited = ', len(visited))

            neigh       = list(set(flatten(neigh)))
            domains.append(np.array(neigh))
    domains = np.array(domains, dtype=object)
    return domains

In [ ]:
if not skip_processing:
    t_start  = 44
    R        = 0.86
    domains  = []
    filename = '../all_domains.sav' 
    for i in np.arange(t_start, len(g)):
        print(i)
        domains.append(get_domains(timestep = i, cutoff = R, all_labels = all_ae_labels))

    pickle.dump(domains, open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
filename  = '../all_domains.sav'
groups    = np.array(pickle.load(open(filename, 'rb')), dtype = object)[-1]
print('total number of domains = ', len(groups))

significant_groups = []                            #ignoring groups with 100 monomers or less
for i in np.arange(len(groups)):
    if len(groups[i]) > 100:
        significant_groups.append(groups[i])
        
len(significant_groups)

In [ ]:
comb = []

for i, array in enumerate(significant_groups[:20]):
    id_    = f'group_{i}'
    id_col = np.full((array.shape[0], 1), id_)
    array  = np.hstack((id_col, obj.folded[significant_groups[i]]))
    comb.append(array)

all_data = np.vstack(comb)

cols = ['Group#', 'x', 'y', 'z']
df = pd.DataFrame(all_data, columns=cols)

df.to_csv('../../03_figures/FigS2/FigS2a.csv')

In [ ]:
fig    = plt.figure()
ax     = fig.add_subplot(111, projection = '3d')
for i in np.arange(10,20): 
    p1 = (obj.folded[significant_groups[i]])
    ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.1, label = 'domain ' + str(i+1))

ax.set_xlabel('$X~(\sigma)$', fontsize = 20)
ax.set_ylabel('$Y~(\sigma)$', fontsize = 20)
ax.set_zlabel('$Z~(\sigma)$', fontsize = 20)
lgnd = ax.legend(fontsize=10)
for handle in lgnd.legendHandles:
    handle.set_sizes([30.0])
plt.savefig('../../03_figures/FigS2/FigS2a.png')

In [ ]:
group_number = 22

def closest_pt(pt, pts):
    closest_index = distance.cdist([pt], pts).argmin()
    return pts[closest_index]

members    = obj.folded[significant_groups[group_number]]
mid_point  = np.average(members, axis = 0)
mid_member = closest_pt(mid_point, members)
print('Coordinates of the cental atom = ', mid_member)

## Q-tensor

In [ ]:
def compute_Q_tensor(all_bondsv):
    
    Q_ab = np.empty(shape = (all_bondsv.shape[0], 3, 3), dtype = np.float64)
    for i, vector in enumerate(all_bondsv):
            Q_ab[i, 0, 0] = 3.0 * vector[0] * vector[0] - 1
            Q_ab[i, 0, 1] = 3.0 * vector[0] * vector[1]
            Q_ab[i, 0, 2] = 3.0 * vector[0] * vector[2]
            Q_ab[i, 1, 0] = 3.0 * vector[1] * vector[0]
            Q_ab[i, 1, 1] = 3.0 * vector[1] * vector[1] - 1
            Q_ab[i, 1, 2] = 3.0 * vector[1] * vector[2]
            Q_ab[i, 2, 0] = 3.0 * vector[2] * vector[0]
            Q_ab[i, 2, 1] = 3.0 * vector[2] * vector[1]
            Q_ab[i, 2, 2] = 3.0 * vector[2] * vector[2] - 1
    Q_ab /= (2.0 * all_bondsv.shape[1])
    return Q_ab

In [ ]:
qt          = []
eignvectors = []
for i in np.arange(len(significant_groups)):
    qt.append(compute_Q_tensor(obj.bonds[significant_groups[i]]))
    
    print('*********', i, 'th group **********')

    s  = sum(qt[i])/len(significant_groups[i])
    print('Added matrix: ')
    print( s)

    eigvals, eigvecs = linlag.eig(s)
    print()

    print('Eigenvalues = ',eigvals)
    print('Eigen vectors :')
    print(eigvecs)
    print('Largest eigenvalue-vector = ', eigvecs[:, np.argmax(eigvals)])
    eignvectors.append(eigvecs[:, np.argmax(eigvals)])

In [ ]:
fig        = plt.figure()
ax         = fig.add_subplot(111, projection = '3d')

p1         = obj.folded[significant_groups[group_number]]
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.3, c = 'r')

mid_point  = np.average(p1, axis = 0)
mid_member = closest_pt(mid_point, p1)
print('Cental atom = ', mid_member)

evd        = np.array(( mid_member, mid_member + 10 * eignvectors[group_number] ))      #Eigen_vector direction

plt.plot ( evd[:,0], evd[:,1], evd[:,2] )

ax.set_xlabel('$X$', fontsize = 30)
ax.set_ylabel('$Y$', fontsize = 30)
ax.set_zlabel('$Z$', fontsize = 30)

### Projection of particles on the plane perpendicular to the eigen vector corresponding to maximum eigenvalue per domain

In [ ]:
members      = obj.folded[significant_groups[group_number]]
members_bond = obj.bonds[significant_groups[group_number]]
eig_vector   = eignvectors[group_number]
np.sum(members_bond[:100] * eig_vector, axis = 1)

In [ ]:
#eig_vector is the orthogonal vector to plane
#we project all the points in a crytal domain on this plane
def projection_on_plane(vectors, e_vec):
    
    proj_on_plane = []
    e_norm        = np.sqrt(sum(e_vec**2)) 
    for i in vectors:
        proj_of_i_on_e_vec = (np.sum (i * e_vec) /e_norm**2) * e_vec
        proj_on_plane.append (i - proj_of_i_on_e_vec)
        
    return np.array(proj_on_plane)

pon = projection_on_plane(obj.folded[significant_groups[group_number]], eig_vector)

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection = '3d')
p1  = pon
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.3, c = 'r')
ax.set_xlabel('$X$', fontsize = 30)
ax.set_ylabel('$Y$', fontsize = 30)
ax.set_zlabel('$Z$', fontsize = 30)

In [ ]:
def orthogonal_vectors(vector):
    # Generating 2 perpendicular vectors (n, m), n = average bond oreintation direction
    vector /= np.linalg.norm(vector)
    m       = np.random.randn(3)                    # take a random vector
    m      -= m.dot(vector) * vector                # make it orthogonal to k
    m      /= np.linalg.norm(m)
    n       = np.cross(vector, m)                   # cross product with k  
    return m, n

m, n        = orthogonal_vectors(eignvectors[group_number])
print(eignvectors[group_number], m, n)
print()
print(np.dot(m, n), np.dot(m, eignvectors[group_number]), np.dot(n, eignvectors[group_number]))

In [ ]:
fig        = plt.figure()
ax         = plt.axes(projection = '3d')
p1         = pon.copy()
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.3)

mid_point  = np.average(p1, axis = 0)
mid_member = closest_pt(mid_point, p1)
print('Cental atom = ', mid_member)

evd        = np.array(( mid_member, mid_member + 7 * m ))
p1         = evd
ax.plot3D(p1[:,0], p1[:,1], p1[:,2], c = 'r')

evd        = np.array(( mid_member, mid_member + 10 * n ))
p1         = evd
ax.plot3D(p1[:,0], p1[:,1], p1[:,2], c = 'b')

evd        = np.array(( mid_member, mid_member + 4 * eignvectors[group_number] ))
p1         = evd
ax.plot3D(p1[:,0], p1[:,1], p1[:,2], c = 'g')

ax.set_xlabel('$X$', fontsize = 30)
ax.set_ylabel('$Y$', fontsize = 30)
ax.set_zlabel('$Z$', fontsize = 30)
#plt.savefig('One_cluster.jpg', dpi = 700)

In [ ]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b            = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v               = np.cross(a, b)
    c               = np.dot(a, b)
    s               = np.linalg.norm(v)
    kmat            = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s**2))
    return rotation_matrix

rot_mat             = rotation_matrix_from_vectors(eignvectors[group_number], np.array([1,0,0]))
trace_rot_mat       = rot_mat.trace()
ang_rot_mat         = np.cos(0.5 * trace_rot_mat - 0.5)
print(rot_mat)
print(eignvectors[group_number], np.matmul(rot_mat, eignvectors[group_number]))

rotated_points      = np.matmul( rot_mat, np.transpose( obj.folded[ significant_groups[ group_number]]))
rotated_ev          = np.matmul( rot_mat, eignvectors[ group_number])

In [ ]:
fig = plt.figure(figsize = (10,6))
ax  = fig.add_subplot(111, projection = '3d')
pon = projection_on_plane(np.transpose(rotated_points), rotated_ev)
p1  = pon
ax.scatter(p1[:,0], p1[:,1], p1[:,2], s = 0.3, c = 'r')
ax.set_xlabel('$X$', fontsize = 30)
ax.set_ylabel('$Y$', fontsize = 30)
ax.set_zlabel('$Z$', fontsize = 30)

fig = plt.figure(figsize = (10,6))
plt.scatter(p1[:,0], p1[:,1], s = 0.3, c = 'r')
fig = plt.figure(figsize = (10,6))
plt.scatter(p1[:,0], p1[:,2], s = 0.3, c = 'r')
fig = plt.figure(figsize = (10,6))
plt.scatter(p1[:,1], p1[:,2], s = 0.3, c = 'r')
plt.xlabel('$Y$', fontsize = 30)
plt.ylabel('$Z$', fontsize = 30)


In [ ]:
plt.rcParams['xtick.labelsize'] = 30
plt.rcParams['ytick.labelsize'] = 30

In [ ]:
# plot 2D histogram using pcolor
fig    = plt.figure(figsize = (10, 10))
p1     = pon[:, 1:3]
size   = 256
w      = 10
c      = [np.mean(p1[:,0]), np.mean(p1[:,1])]

signal = plt.hist2d(p1[:,0], p1[:,1], range = ((c[0] - w, c[0] + w), (c[1] - w, c[1] + w)), bins = size, cmap = plt.cm.twilight)
np.savetxt('../../03_figures/FigS2/FigS2b.csv', p1, delimiter=",")
plt.xlabel('$Y~(\sigma)$', fontsize = 35)
plt.ylabel('$Z~(\sigma)$', fontsize = 35)
plt.savefig('../../03_figures/FigS2/FigS2b')


In [ ]:
from matplotlib.patches import Circle

def plot_spectrum(im_fft):
    from matplotlib.colors import LogNorm # A logarithmic colormap
    qmax = 2.0*np.pi/(2*w/(im_fft.shape[0]/2))
    print ("qmax = ",qmax)
    plt.imshow(np.abs(im_fft), norm = LogNorm(vmin = 10), extent = (-qmax,qmax,-qmax,qmax))
    plt.gca().add_patch(Circle((0,0), radius=2.0*np.pi/(10./11.9), color="red", fill=False))
    #plt.colorbar()

dft       = np.fft.fft2(signal[0])
dft_shift = np.fft.fftshift(dft)
mag       = np.abs(dft_shift)
ang       = np.angle(dft_shift)

np.savetxt('../../03_figures/FigS2/FigS2c.csv', dft_shift, delimiter=",")

plt.figure(figsize = (10,10))
plot_spectrum(dft_shift)
#plt.title ('Fourier transform of the crystal domain', fontdict = {'fontsize' : 15})
#plt.title('Fourier transform')
plt.xlabel('$2π~/~\lambda_y$', fontsize = 30)
plt.ylabel('$2π~/~\lambda_z$', fontsize = 30)
plt.savefig('../../03_figures/FigS2/FigS2c')

In [ ]:
2.0*np.pi/(10./11.9)